# MNC-KAIST
## Neural Network
### 2021 / 12 / 24
Copyright @ cb_park@korea.ac.kr (Cheonbok Park), joonleesky@kaist.ac.kr (Hojoon Lee)

## MNIST Feed-Forward Neural Network

In [1]:
import torch 
import torch.nn as nn # neural network 
import torch.nn.functional as F # 각종 activation 함수
import torchvision # 이미지 관련 처리, Pretrained Model 관련된 Package 입니다. 
import torchvision.datasets as vision_dsets
import torchvision.transforms as T # 이미지 처리 (Vison) 관련된 transformation이 정의 되어 있습니다.
import torch.optim as optim # pytorch 에서 정의한 수 많은 optimization function 들이 들어 있습니다.
from torch.utils import data

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

## Data Loader 불러오기

In [2]:
def MNIST_DATA(root='./data',train =True,transforms=None ,download =True,batch_size = 32,num_worker = 1):
    print ("[+] Get the MNIST DATA")
    """
    torchvision.dataset 에는 우리가 많이 사용하는 데이터들을 쉽게 사용할 수 있도록 되어 있습니다. 
    Mchine Learning 에서 Hello world 라고 불리는 Mnist 데이터를 사용해 보겠습니다. 
    """
    mnist_train = vision_dsets.MNIST(root = root,  #root 는 데이터의 저장 위치 입니다. 
                                    train = True, #Train 은 이 데이터가 train 데이터인지 아닌지에 대한 정보입니다. 
                                    transform = T.ToTensor(), # 얻어낸 데이터를 pytorch가 계산 할 수 있는 Tensor 로 변환해 줍니다. 
                                    download = True)  # 데이터를 다운로드 할지 여부를 물어봅니다. 
    mnist_test = vision_dsets.MNIST(root = root,
                                    train = False,  # Test Data를 가져오기에 Train =False 를 줘야 합니다. 
                                    transform = T.ToTensor(),
                                    download = True)
    """
    Data Loader 는 데이터와 batch size의 정보를 바탕으로 매 iteration 마다 주어진 데이터를 원하는 batch size 만큼 반환해주는 iterator입니다. 
    * Practical Guide : Batch size 는 어느정도가 좋나요? -- 클 수록 좋다는 소리가 있습니다. 하지만 gpu memeory 사이즈 한계에 의해 기본적으로 batch size 가 
      커질 수록 학습에 사용되는 gpu memory 사이즈가 큽니다. (Activation map을 저장해야 하기 때문입니다.) 기본적으로 2의 배수로 저장하는 것이 좋습니다.(Bit size 관련) 
    """
    trainDataLoader = data.DataLoader(dataset = mnist_train,  # DataSet은 어떤 Data를 제공해 줄지에 대한 정보입니다. 여기서는 Training DATA를 제공합니다. 
                                      batch_size = batch_size, # batch size 정보를 꼭 줘야 합니다. 한 Batch 당 몇 개의 Data 를 제공할지에 대한 정보입니다. 
                                      shuffle =True, # Training의 경우 Shuffling 을 해주는 것이 성능에 지대한 영향을 끼칩니다. 꼭 True 를 줘야 합니다. 
                                      num_workers = 1) # num worker의 경우 데이터를 로드하는데 worker를 얼마나 추가하겠는가에 대한 정보입니다. 

    testDataLoader = data.DataLoader(dataset = mnist_test, # Test Data Loader 이므로 Test Data를 인자로 전달해줍니다.
                                    batch_size = batch_size, # 마찬가지로 Batch size 를 넣어줍니다. 
                                    shuffle = False, # shuffling 이 굳이 필요하지 않으므로 false를 줍니다. 
                                    num_workers = 1) #
    print ("[+] Finished loading data & Preprocessing")
    return mnist_train,mnist_test,trainDataLoader,testDataLoader

In [3]:
trainDset,testDset,trainDataLoader,testDataLoader= MNIST_DATA(batch_size = 32)  # Data Loader 를 불러 옵니다. 

[+] Get the MNIST DATA
[+] Finished loading data & Preprocessing


## Trainer 정의하기

In [ ]:
# gpu를 못 쓴다면 .cuda()를 빼거나 대신 .to('cpu')를 넣자

In [10]:
class Trainer():
    def __init__(self, trainloader, testloader, net, optimizer, criterion):
        """
        trainloader: train data의 loader 입니다
        testloader: test data의 loader 입니다
        net: 학습시킬 모델입니다
        optimizer: 모델의 파라미터를 업데이트할 최적화 함수입니다
        criterion: 모델의 loss function 입니다.
        """
        self.trainloader = trainloader
        self.testloader = testloader
        self.net = net
        self.optimizer = optimizer
        self.criterion = criterion
        
    def train(self, epoch = 1):
        """
        epoch: 전체 학습 데이터의 사용횟수입니다.
        """
        self.net.train()
        for e in range(epoch):
            running_loss = 0.0 # running loss를 저장하기 위한 변수입니다. 
            for i, data in enumerate(self.trainloader, 0): # 한 Epoch 만큼 돕니다. 매 iteration 마다 정해진 Batch size 만큼 데이터를 뱉습니다. 
                # get the inputs
                inputs, labels = data # DataLoader iterator의 반환 값은 input_data 와 labels의 튜플 형식입니다. 
                inputs = inputs.to('cpu')
                labels = labels.to('cpu')
                # zero the parameter gradients
                self.optimizer.zero_grad()    #  현재 기존의 backprop을 계산하기 위해서 저장했던 activation buffer 를 비웁니다. 
                                              #  Q) 이걸 안 한다면?

                # forward + backward + optimize
                outputs = self.net(inputs) # input 을 넣은 위 network 로 부터 output 을 얻어냅니다. 
                loss = self.criterion(outputs, labels) # loss fucntion에 주어진 target과 output 의 score를 계산하여 반환합니다. 
                loss.backward() # * Scalar Loss value를 Backward() 해주게 되면 주어진 loss값을 바탕으로 backpropagation이 진행됩니다. 
                self.optimizer.step() # 계산된 Backprop 을 바탕으로 optimizer가 gradient descenting 을 수행합니다. 

                # print statistics
                running_loss += loss.item()
                if (i+1) % 500 == 0:    # print every 2000 mini-batches
                    print('[%d, %5d] loss: %.3f' % (e + 1, i + 1, running_loss / 500))
                    running_loss = 0.0

        print('Finished Training')
        
    def test(self):
        with torch.no_grad():
            self.net.eval() # Eval Mode 왜 해야 할까요?  
                            # --> nn.Dropout BatchNorm 등의 Regularization 들이 test 모드로 들어가게 되기 때문입니다. 
            test_loss = 0
            correct = 0
            for inputs, labels in self.testloader:
                inputs = inputs.to('cpu')
                labels = labels.to('cpu')
                output = self.net(inputs) 
                pred = output.max(1, keepdim=True)[1] # get the index of the max 
                correct += pred.eq(labels.view_as(pred)).sum().item() # 정답 데이터의 갯수를 반환합니다. 

                test_loss /= len(self.testloader.dataset)
            print('\nTest set:  Accuracy: {}/{} ({:.0f}%)\n'.
                    format(correct, len(self.testloader.dataset),
                    100.* correct / len(self.testloader.dataset)))

In [ ]:
# pytorch
# B * C * H * W 
# image ... 필기 좀 하자...ㅡ.ㅡ;; 

## Network 만들어보기

![activation](./imgs/activation.png)
출처: cs231n

### (1) 2-Layer Network + Sigmoid

- Input: (28 * 28)
- Hidden dimension: 30
- Output dimension: 10
- activation: sigmoid
- Optimizer: SGD
- Loss: Cross-Entropy

In [11]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 
                            # 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.sigmoid(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [12]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [13]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [14]:
trainer.train(epoch = 4)

C:\Users\ASUS\anaconda3\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   500] loss: 2.315
[1,  1000] loss: 2.297
[1,  1500] loss: 2.277
[2,   500] loss: 2.251
[2,  1000] loss: 2.239
[2,  1500] loss: 2.226
[3,   500] loss: 2.201
[3,  1000] loss: 2.187
[3,  1500] loss: 2.170
[4,   500] loss: 2.141
[4,  1000] loss: 2.125
[4,  1500] loss: 2.104
Finished Training


In [15]:
trainer.test()


Test set:  Accuracy: 5336/10000 (53%)



### (2) 2-Layer Network + ReLU

- Input: (28 * 28)
- Hidden dimension: 30
- Output dimension: 10
- activation: relu
- Optimizer: SGD
- Loss: Cross-Entropy

In [17]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [18]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [19]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [20]:
trainer.train(epoch = 4)

[1,   500] loss: 2.276
[1,  1000] loss: 2.185
[1,  1500] loss: 2.070
[2,   500] loss: 1.837
[2,  1000] loss: 1.693
[2,  1500] loss: 1.545
[3,   500] loss: 1.291
[3,  1000] loss: 1.170
[3,  1500] loss: 1.063
[4,   500] loss: 0.917
[4,  1000] loss: 0.853
[4,  1500] loss: 0.799
Finished Training


In [21]:
trainer.test()


Test set:  Accuracy: 8400/10000 (84%)



#### Q) Activation별로 성능차이가 존재하나요? 존재한다면 왜 존재할까요?

#### Ans)

### (3) 3-Layer Network + Sigmoid

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: sigmoid
- Optimizer: SGD
- Loss: Cross-Entropy

In [22]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.sigmoid(x)
        x = self.fc1(x)
        x = F.sigmoid(x)
        x = self.fc2(x)
        return x

In [23]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [24]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [25]:
trainer.train(epoch = 4)

[1,   500] loss: 2.323
[1,  1000] loss: 2.310
[1,  1500] loss: 2.304
[2,   500] loss: 2.301
[2,  1000] loss: 2.300
[2,  1500] loss: 2.300
[3,   500] loss: 2.299
[3,  1000] loss: 2.298
[3,  1500] loss: 2.298
[4,   500] loss: 2.298
[4,  1000] loss: 2.298
[4,  1500] loss: 2.296
Finished Training


In [26]:
trainer.test()


Test set:  Accuracy: 1135/10000 (11%)



#### Q) 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

### (4) 3-Layer Network + ReLU

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- Optimizer: SGD
- Loss: Cross-Entropy

In [27]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [28]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [29]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [30]:
trainer.train(epoch = 4)

[1,   500] loss: 2.297
[1,  1000] loss: 2.287
[1,  1500] loss: 2.274
[2,   500] loss: 2.242
[2,  1000] loss: 2.215
[2,  1500] loss: 2.181
[3,   500] loss: 2.093
[3,  1000] loss: 2.024
[3,  1500] loss: 1.929
[4,   500] loss: 1.734
[4,  1000] loss: 1.596
[4,  1500] loss: 1.463
Finished Training


In [31]:
trainer.test()


Test set:  Accuracy: 6361/10000 (64%)



#### Q) (2)와 비교했을 때 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

#### Q) Activation을 사용하지 않는다면 어떤 일이 일어날까요?

#### Ans) 

### (5) 3-Layer Network + Sine

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: Sine
- Optimizer: SGD
- Loss: Cross-Entropy

In [32]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = torch.sin(x)
        x = self.fc1(x)
        x = torch.sin(x)
        x = self.fc2(x)
        return x

In [33]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.SGD(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [34]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [35]:
trainer.train(epoch = 4)

[1,   500] loss: 2.276
[1,  1000] loss: 2.214
[1,  1500] loss: 2.149
[2,   500] loss: 1.994
[2,  1000] loss: 1.886
[2,  1500] loss: 1.763
[3,   500] loss: 1.546
[3,  1000] loss: 1.429
[3,  1500] loss: 1.338
[4,   500] loss: 1.175
[4,  1000] loss: 1.125
[4,  1500] loss: 1.055
Finished Training


In [36]:
trainer.test()


Test set:  Accuracy: 7742/10000 (77%)



#### Q) (2)와 비교했을 때 학습이 원할하게 이루어지나요? 이루어지지 않는다면 왜 이루어지지 않을까요?

#### Ans)

## Let's Change our Optimizer

![Adam](./imgs/adam.jpeg)
출처: 하용호, 자습해도 모르겠던 딥러닝, 머리속에 인스톨 시켜드립니다

### (6) 3-Layer Network + ReLU + Adam

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- Optimizer: Adam
- Loss: Cross-Entropy

In [37]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [38]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [39]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [40]:
trainer.train(epoch = 4)

[1,   500] loss: 0.648
[1,  1000] loss: 0.306
[1,  1500] loss: 0.263
[2,   500] loss: 0.186
[2,  1000] loss: 0.179
[2,  1500] loss: 0.166
[3,   500] loss: 0.131
[3,  1000] loss: 0.127
[3,  1500] loss: 0.131
[4,   500] loss: 0.106
[4,  1000] loss: 0.106
[4,  1500] loss: 0.103
Finished Training


In [41]:
trainer.test()


Test set:  Accuracy: 9659/10000 (97%)



### (7) 2-Layer Network + ReLU + Adam

- Input: (28 * 28)
- Hidden dimension: (30)
- Output dimension: 10
- activation: relu
- Optimizer: Adam
- Loss: Cross-Entropy

In [42]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [43]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [44]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [45]:
trainer.train(epoch = 4)

[1,   500] loss: 0.685
[1,  1000] loss: 0.314
[1,  1500] loss: 0.273
[2,   500] loss: 0.224
[2,  1000] loss: 0.212
[2,  1500] loss: 0.207
[3,   500] loss: 0.178
[3,  1000] loss: 0.175
[3,  1500] loss: 0.167
[4,   500] loss: 0.152
[4,  1000] loss: 0.154
[4,  1500] loss: 0.145
Finished Training


In [46]:
trainer.test()


Test set:  Accuracy: 9590/10000 (96%)



## Batch-Normalization

![normalization](./imgs/normalization.png)
출처: Andrew Ng, Deep Learning

### (8) 2-Layer Network + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Hidden dimension: (30)
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [47]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__() # nn.Module 생성자 호출 Q) 왜 필요할까요?
        # an affine operation: y = Wx + b
        self.fc0 = nn.Linear(28*28,30)
        self.bn0 = nn.BatchNorm1d(30) # BatchNorm 
        self.fc1 = nn.Linear(30, 10)

    def forward(self, x):
        x = x.view(-1,28*28) # x.view함수는 주어진 인자의 크기로 해당 데이터의 크기를 반환합니다. 즉, (Batch_size,28,28) --> (Batch_size,28*28)로 변환합니다.
        x = self.fc0(x) # 28*28 -> 30 
        x = self.bn0(x) # BatchNorm 
        x = F.relu(x) # Activation function 을 수행합니다.
        x = self.fc1(x)  # 30 -> 10 으로 10개의 Class에 대한 logit 값을 호출합니다. 
        return x

In [48]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [49]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [50]:
trainer.train(epoch = 4)

[1,   500] loss: 0.728
[1,  1000] loss: 0.333
[1,  1500] loss: 0.267
[2,   500] loss: 0.216
[2,  1000] loss: 0.208
[2,  1500] loss: 0.199
[3,   500] loss: 0.167
[3,  1000] loss: 0.170
[3,  1500] loss: 0.165
[4,   500] loss: 0.143
[4,  1000] loss: 0.149
[4,  1500] loss: 0.147
Finished Training


In [51]:
trainer.test()


Test set:  Accuracy: 9629/10000 (96%)



### (9) 3-Layer Network + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Hidden dimension: (50, 30)
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [52]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.fc0 = nn.Linear(28*28,50) # Layer 1
        self.bn0 = nn.BatchNorm1d(50) # BatchNorm 1 
        self.fc1 = nn.Linear(50, 30) # Layer 2
        self.bn1 = nn.BatchNorm1d(30) # BatchNorm 2
        self.fc2 = nn.Linear(30, 10) # Layer 3

    def forward(self, x):
      
        x = x.view(-1,28*28)
        x = self.fc0(x)
        x = self.bn0(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [53]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [54]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [55]:
trainer.train(epoch = 4)

[1,   500] loss: 0.636
[1,  1000] loss: 0.264
[1,  1500] loss: 0.205
[2,   500] loss: 0.158
[2,  1000] loss: 0.147
[2,  1500] loss: 0.133
[3,   500] loss: 0.113
[3,  1000] loss: 0.122
[3,  1500] loss: 0.115
[4,   500] loss: 0.099
[4,  1000] loss: 0.092
[4,  1500] loss: 0.100
Finished Training


In [56]:
trainer.test()


Test set:  Accuracy: 9745/10000 (97%)



#### Q) Batch-norm을 적용하기 전과 후의 성능이 어떤가요? 

#### Ans) 성능이 좋아지지는 않는다고 하지만, training에 들어가는 속도가 빠르다

#### Q) Batch-norm을 적용한 이후의 2-layer와 3-layer network의 성능은 어떤 변화 양상을 보여주었나요?

#### Ans) Layer 하나가 추가됨에 따라 학습 속도가 느려졌다

In [58]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mnist_net)

41250

In [60]:
[i.numel() for i in mnist_net.parameters()]

[39200, 50, 50, 50, 1500, 30, 30, 30, 300, 10]

In [62]:
[i.numel() for i in mnist_net.parameters() if i.requires_grad]

[39200, 50, 50, 50, 1500, 30, 30, 30, 300, 10]

#### Q) 파라미터 수가 많으면 어떠한 문제점이 발생할까요?

#### Ans)

## Let's use Convolution Layer

### Convolution Operation

![Convolution](./imgs/Conv.png)

#### Q) (H, W, C1)의 이미지에 C2개의 (F * F) filter를 stride 크기 S로 convolution하면 output size가 어떻게 될까요?
#### Ans) 

### (10) 3-Layer Network (Conv+Fc) + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Conv: 8 (6 * 6) filter with stride=2 
- Hidden dimension: 8 * 12 * 12
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [66]:
# B * 1 * 28 * 28
# B * (1 * 28 * 28) ==> Input of the FC layer
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.BatchNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.fc = nn.Linear(8*12*12, 10) # Layer 2 (왜 hidden이 8 * 12 * 12 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        # print(x.shape, 'The output of layer1') # 확인용 
        x = F.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [67]:
mnist_net = MNIST_Net().to('cpu') # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [68]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [69]:
trainer.train(epoch = 4)

[1,   500] loss: 0.398
[1,  1000] loss: 0.187
[1,  1500] loss: 0.140
[2,   500] loss: 0.091
[2,  1000] loss: 0.086
[2,  1500] loss: 0.081
[3,   500] loss: 0.064
[3,  1000] loss: 0.062
[3,  1500] loss: 0.063
[4,   500] loss: 0.051
[4,  1000] loss: 0.051
[4,  1500] loss: 0.054
Finished Training


In [70]:
trainer.test()


Test set:  Accuracy: 9814/10000 (98%)



In [71]:
count_parameters(mnist_net)

11842

#### Q) Convolution operation을 사용했을 때 성능과 파라미터가 어떻게 변했나요? 왜 이런 결과가 나왔을까요?

#### Ans) 정확도가 매우 높아졌다.

### (11) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Batch-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: batch-norm
- Optimizer: Adam
- Loss: Cross-Entropy

### Pooling Operation

![Pooling](./imgs/Pool.png)

In [52]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.BatchNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [53]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [54]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [55]:
trainer.train(epoch = 4)

[1,   500] loss: 0.551
[1,  1000] loss: 0.194
[1,  1500] loss: 0.149
[2,   500] loss: 0.112
[2,  1000] loss: 0.106
[2,  1500] loss: 0.097
[3,   500] loss: 0.085
[3,  1000] loss: 0.077
[3,  1500] loss: 0.084
[4,   500] loss: 0.072
[4,  1000] loss: 0.070
[4,  1500] loss: 0.076
Finished Training


In [56]:
trainer.test()


Test set:  Accuracy: 9783/10000 (98%)



In [57]:
count_parameters(mnist_net)

3202

#### Q) Pooling을 쓰기 전과 후에 성능과 파라미터 수가 어떻게 변했나요? 왜 이런 결과가 나왔을까요?

#### Ans)

### (12) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Instance-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Instance-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [49]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.InstanceNorm2d(8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [50]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [51]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [52]:
trainer.train(epoch = 4)

[1,   500] loss: 0.594
[1,  1000] loss: 0.238
[1,  1500] loss: 0.189
[2,   500] loss: 0.127
[2,  1000] loss: 0.122
[2,  1500] loss: 0.111
[3,   500] loss: 0.096
[3,  1000] loss: 0.090
[3,  1500] loss: 0.085
[4,   500] loss: 0.077
[4,  1000] loss: 0.077
[4,  1500] loss: 0.080
Finished Training


In [53]:
trainer.test()


Test set:  Accuracy: 9806/10000 (98%)



### (13) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Layer-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Layer-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [77]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.LayerNorm([8,12,12])  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)

        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [78]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [79]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [80]:
trainer.train(epoch = 4)

[1,   500] loss: 0.533
[1,  1000] loss: 0.200
[1,  1500] loss: 0.148
[2,   500] loss: 0.104
[2,  1000] loss: 0.095
[2,  1500] loss: 0.093
[3,   500] loss: 0.080
[3,  1000] loss: 0.077
[3,  1500] loss: 0.071
[4,   500] loss: 0.066
[4,  1000] loss: 0.060
[4,  1500] loss: 0.065
Finished Training


In [81]:
trainer.test()


Test set:  Accuracy: 9819/10000 (98%)



### (14) 3-Layer Network (Conv+Pool+Fc) + ReLU + Adam + Grouph-Norm

- Input: (28 * 28)
- Conv: 8 (7 * 7) filter with stride=2 
- Pool: 2 * 2
- Hidden dimension: 8 * 6 * 6
- Output dimension: 10
- activation: relu
- normalization: Group-norm
- Optimizer: Adam
- Loss: Cross-Entropy

In [103]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()
        self.conv0 = nn.Conv2d(in_channels = 1,
                               out_channels = 8,
                               kernel_size = 6,
                               stride = 2) # Layer 1
        self.conv0_bn = nn.GroupNorm(num_groups = 4, num_channels = 8)  # Image에서는 2d batchnorm이 사용됩니다
        self.pool0 = nn.MaxPool2d(2)
        self.fc = nn.Linear(8*6*6, 10) # Layer 2 (왜 input이 8 * 11 * 11 일까요?)

    def forward(self, x):
        x = self.conv0(x)
        x = self.conv0_bn(x)
        x = F.relu(x)
        x = self.pool0(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

In [104]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [105]:
trainer = Trainer(trainloader = trainDataLoader,
                  testloader = testDataLoader,
                  net = mnist_net,
                  criterion = criterion,
                  optimizer = optimizer)

In [106]:
trainer.train(epoch = 4)

[1,   500] loss: 0.549
[1,  1000] loss: 0.221
[1,  1500] loss: 0.159
[2,   500] loss: 0.121
[2,  1000] loss: 0.108
[2,  1500] loss: 0.098
[3,   500] loss: 0.084
[3,  1000] loss: 0.080
[3,  1500] loss: 0.082
[4,   500] loss: 0.068
[4,  1000] loss: 0.072
[4,  1500] loss: 0.069
Finished Training


In [107]:
trainer.test()


Test set:  Accuracy: 9793/10000 (98%)



## Let's Do It: 10000개 이하의 파라미터로 99%의 성능을 달성해 볼까요?

In [72]:
class MNIST_Net(nn.Module):
    def __init__(self):
        super(MNIST_Net, self).__init__()

    def forward(self, x):
        return x

In [ ]:
mnist_net = MNIST_Net().cuda() # 생성한 뉴럴넷 Instance를 생성하고 빠른 학습을 위해 cuda 에 올립니다. 
criterion = nn.CrossEntropyLoss() # Loss Function을 정의 합니다. 여기서는 cross entrophy loss 를 사용합니다. 
optimizer = optim.Adam(mnist_net.parameters(), lr=0.001) # optimizer는 이와 같이 training 할 Parameter와 learning rate를 인자로 줍니다. 

In [ ]:
train_network(mnist_net,optimizer,trainDataLoader)

In [ ]:
test(mnist_net,testDataLoader)